In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import warnings
from tqdm import tqdm
import os
from pathlib import Path
# Define the device
device = "cuda" if torch.cuda.is_available() else "mps" if  torch.backends.mps.is_available() else "cpu" # torch.has_mps or
print("Using device:", device)
if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
elif (device == 'mps'):
    print(f"Device name: <mps>")
else:
    print("NOTE: If you have a GPU, consider using it for training.")
    print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
    print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
device = torch.device(device)

Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        output = model(ESP_val_df_enzy,ESP_val_df_smiles)

        loss = loss_fn(output, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (output > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(output.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


### esm2 480 + MolFormer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_480')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(1248, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_480_MolFormer.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_480_MolFormer.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_480_MolFormer.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_480_MolFormer.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_480_MolFormer.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_480_MolFormer.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_480_MolFormer.pt')
    y_val = torch.load('ESP_val_df_label_esm2_480_MolFormer.pt')
    y_test = torch.load('ESP_test_df_label_esm2_480_MolFormer.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([50077, 480]) torch.Size([5422, 480]) torch.Size([13336, 480])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_simple.pt")

Epoch 0: 100%|██████████| 3130/3130 [00:18<00:00, 169.04batch/s, train_loss=tensor(0.5287, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5625066940499618 ACC=  0.7373662855035042 bacc=  0.5026699661940274 precision=  0.8888888888888888 specificity=  0.9997494362315209 sensitivity=  0.005590496156533892 recall=  0.005590496156533892 MCC=  0.057817082194654645 AUC=  0.6071429759586604 f1=  0.011111111111111112
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5625); Accuracy (73.74)


Epoch 1: 100%|██████████| 3130/3130 [00:17<00:00, 179.61batch/s, train_loss=tensor(0.9715, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5413573072195756 ACC=  0.7388417558096644 bacc=  0.5054652142722943 precision=  0.9411764705882353 specificity=  0.9997494362315209 sensitivity=  0.011180992313067784 recall=  0.011180992313067784 MCC=  0.08617386270086295 AUC=  0.6653615813778067 f1=  0.02209944751381215
Epoch: 1 / 500, ############## the best accuracy in val  73.7366 at Epoch: 0  ##############
Performance in Val: Loss: (0.5414); Accuracy (73.88)


Epoch 2: 100%|██████████| 3130/3130 [00:17<00:00, 183.24batch/s, train_loss=tensor(0.5002, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.3705708014138852 ACC=  0.882146809295463 bacc=  0.8388119425240683 precision=  0.7942050520059435 specificity=  0.9305938361312954 sensitivity=  0.7470300489168413 recall=  0.7470300489168413 MCC=  0.6913687872709247 AUC=  0.9257166850430939 f1=  0.7698955707598127
Epoch: 497 / 500, ############## the best accuracy in val  88.1594 at Epoch: 488  ##############
Performance in Val: Loss: (0.3706); Accuracy (88.21)


Epoch 498: 100%|██████████| 3130/3130 [00:16<00:00, 184.24batch/s, train_loss=tensor(0.1314, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.3831557447987383 ACC=  0.8764293618590926 bacc=  0.8248426184631703 precision=  0.7956487956487956 specificity=  0.9341017288900025 sensitivity=  0.7155835080363382 recall=  0.7155835080363382 MCC=  0.6730334488307226 AUC=  0.9257785818230782 f1=  0.753495217071376
Epoch: 498 / 500, ############## the best accuracy in val  88.2147 at Epoch: 497  ##############
Performance in Val: Loss: (0.3832); Accuracy (87.64)


Epoch 499: 100%|██████████| 3130/3130 [00:17<00:00, 182.20batch/s, train_loss=tensor(0.1245, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.3600307702861079 ACC=  0.8753227591294725 bacc=  0.8451585949588531 precision=  0.7548953409858203 specificity=  0.9090453520420947 sensitivity=  0.7812718378756115 recall=  0.7812718378756115 MCC=  0.6828552019324295 AUC=  0.9240662209748314 f1=  0.7678571428571429
Epoch: 499 / 500, ############## the best accuracy in val  88.2147 at Epoch: 497  ##############
Performance in Val: Loss: (0.3600); Accuracy (87.53)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_simple.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_480_simple.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.4304710984694272 ACC=  0.8640521895620876 bacc=  0.805808129956973 precision=  0.7749434937035841 specificity=  0.9290224032586558 sensitivity=  0.6825938566552902 recall=  0.6825938566552902 MCC=  0.6382044463698018 AUC=  0.907304518882548 f1=  0.7258430364433691


(0.4304710984694272, 0.8640521895620876, 0.805808129956973)

### esm2 1280 + MolFormer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_1280')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(2048, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_1280.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_1280.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_1280.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_1280.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_1280.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_1280.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_1280.pt')
    y_val = torch.load('ESP_val_df_label_esm2_1280.pt')
    y_test = torch.load('ESP_test_df_label_esm2_1280.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([50077, 1280]) torch.Size([5422, 1280]) torch.Size([13336, 1280])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_1280_simple.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.47166643466736535 ACC=  0.8829484103179364 bacc=  0.8388143001791055 precision=  0.7973836324916337 specificity=  0.9321792260692464 sensitivity=  0.7454493742889647 recall=  0.7454493742889647 MCC=  0.6928043880289186 AUC=  0.9255472799353089 f1=  0.7705424077612818


(0.47166643466736535, 0.8829484103179364, 0.8388143001791055)

### esm2 2560 + MolFormer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(3328, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_simple_concatenate.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:18<00:00, 168.58batch/s, train_loss=tensor(0.6408, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.559140573327358 ACC=  0.7392109649935173 bacc=  0.5061899716578815 precision=  0.9473684210526315 specificity=  0.9997483643683945 sensitivity=  0.01263157894736842 recall=  0.01263157894736842 MCC=  0.09214506514779681 AUC=  0.622010171377109 f1=  0.02493074792243767
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5591); Accuracy (73.92)


Epoch 1: 100%|██████████| 3118/3118 [00:17<00:00, 176.34batch/s, train_loss=tensor(0.6681, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5264421592538173 ACC=  0.7403222819040564 bacc=  0.5121212442278318 precision=  0.6949152542372882 specificity=  0.9954705586311021 sensitivity=  0.028771929824561403 recall=  0.028771929824561403 MCC=  0.10277841755037978 AUC=  0.6974199842838096 f1=  0.05525606469002695
Epoch: 1 / 500, ############## the best accuracy in val  73.9211 at Epoch: 0  ##############
Performance in Val: Loss: (0.5264); Accuracy (74.03)


Epoch 2: 100%|██████████| 3118/3118 [00:17<00:00, 178.31batch/s, train_loss=tensor(0.5969, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.5777890068956616 ACC=  0.9125764030375996 bacc=  0.882323612251565 precision=  0.845540246555475 specificity=  0.9464016104680423 sensitivity=  0.8182456140350877 recall=  0.8182456140350877 MCC=  0.7728359579316104 AUC=  0.9496001200787576 f1=  0.8316690442225393
Epoch: 497 / 500, ############## the best accuracy in val  92.0911 at Epoch: 306  ##############
Performance in Val: Loss: (0.5778); Accuracy (91.26)


Epoch 498: 100%|██████████| 3118/3118 [00:17<00:00, 175.66batch/s, train_loss=tensor(0.1118, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.7166971069490135 ACC=  0.9099833302463419 bacc=  0.8686340158398007 precision=  0.8648018648018648 specificity=  0.9562154001006542 sensitivity=  0.7810526315789473 recall=  0.7810526315789473 MCC=  0.762659949229163 AUC=  0.9494608816959359 f1=  0.8207964601769911
Epoch: 498 / 500, ############## the best accuracy in val  92.0911 at Epoch: 306  ##############
Performance in Val: Loss: (0.7167); Accuracy (91.00)


Epoch 499: 100%|██████████| 3118/3118 [00:18<00:00, 173.12batch/s, train_loss=tensor(0.0129, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.6999304321335925 ACC=  0.9125764030375996 bacc=  0.8708455840153984 precision=  0.8731401722787784 specificity=  0.9592350276799195 sensitivity=  0.7824561403508772 recall=  0.7824561403508772 MCC=  0.7692987534539127 AUC=  0.9509925039069742 f1=  0.8253145817912657
Epoch: 499 / 500, ############## the best accuracy in val  92.0911 at Epoch: 306  ##############
Performance in Val: Loss: (0.6999); Accuracy (91.26)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_simple_concatenate.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_simple.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.605212362124922 ACC=  0.8958082932692307 bacc=  0.8509763691461305 precision=  0.8332810047095761 specificity=  0.9458273821669048 sensitivity=  0.7561253561253561 recall=  0.7561253561253561 MCC=  0.7249760448226469 AUC=  0.935724684944232 f1=  0.7928304705003735


(0.605212362124922, 0.8958082932692307, 0.8509763691461305)

### different dropout location

in our paper, we use the following results, this dropout location is more common.

### esm2 480 + MolFormer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_480')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(1248, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)

        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_480_MolFormer.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_480_MolFormer.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_480_MolFormer.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_480_MolFormer.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_480_MolFormer.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_480_MolFormer.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_480_MolFormer.pt')
    y_val = torch.load('ESP_val_df_label_esm2_480_MolFormer.pt')
    y_test = torch.load('ESP_test_df_label_esm2_480_MolFormer.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([50077, 480]) torch.Size([5422, 480]) torch.Size([13336, 480])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_simple.pt")

Epoch 0: 100%|██████████| 3130/3130 [00:18<00:00, 172.90batch/s, train_loss=tensor(0.7019, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5615437252331624 ACC=  0.7375507192917743 bacc=  0.5088465994679503 precision=  0.5645161290322581 specificity=  0.9932347782510649 sensitivity=  0.02445842068483578 recall=  0.02445842068483578 MCC=  0.07334808874317308 AUC=  0.6234953698231923 f1=  0.04688546550569323
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5615); Accuracy (73.76)


Epoch 1: 100%|██████████| 3130/3130 [00:18<00:00, 171.83batch/s, train_loss=tensor(0.5061, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5352895368868622 ACC=  0.7393950571744744 bacc=  0.5067375564271883 precision=  0.9090909090909091 specificity=  0.9994988724630418 sensitivity=  0.013976240391334731 recall=  0.013976240391334731 MCC=  0.09342968218860954 AUC=  0.6692523236681556 f1=  0.02752924982794219
Epoch: 1 / 500, ############## the best accuracy in val  73.7551 at Epoch: 0  ##############
Performance in Val: Loss: (0.5353); Accuracy (73.94)


Epoch 2: 100%|██████████| 3130/3130 [00:18<00:00, 172.65batch/s, train_loss=tensor(0.7966, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.38751365047959185 ACC=  0.8690520103282922 bacc=  0.8586048168126713 precision=  0.7154811715481172 specificity=  0.8807316462039589 sensitivity=  0.8364779874213837 recall=  0.8364779874213837 MCC=  0.6843859804393891 AUC=  0.9171284411589248 f1=  0.7712628865979383
Epoch: 497 / 500, ############## the best accuracy in val  88.2516 at Epoch: 358  ##############
Performance in Val: Loss: (0.3875); Accuracy (86.91)


Epoch 498: 100%|██████████| 3130/3130 [00:18<00:00, 173.40batch/s, train_loss=tensor(0.2473, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.39138756432552657 ACC=  0.8758760604942826 bacc=  0.8471033094903786 precision=  0.7540214477211796 specificity=  0.9080430969681784 sensitivity=  0.7861635220125787 recall=  0.7861635220125787 MCC=  0.6851233106164298 AUC=  0.91959398163688 f1=  0.7697570988710231
Epoch: 498 / 500, ############## the best accuracy in val  88.2516 at Epoch: 358  ##############
Performance in Val: Loss: (0.3914); Accuracy (87.59)


Epoch 499: 100%|██████████| 3130/3130 [00:18<00:00, 170.82batch/s, train_loss=tensor(0.1393, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.3989415042549454 ACC=  0.8775359645887126 bacc=  0.8379211366735182 precision=  0.7757009345794392 specificity=  0.9218241042345277 sensitivity=  0.7540181691125087 recall=  0.7540181691125087 MCC=  0.6820811848647543 AUC=  0.9216448049340926 f1=  0.7647058823529412
Epoch: 499 / 500, ############## the best accuracy in val  88.2516 at Epoch: 358  ##############
Performance in Val: Loss: (0.3989); Accuracy (87.75)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_simple.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_480_simple_new.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.42307043597849653 ACC=  0.8642021595680864 bacc=  0.8125741735771764 precision=  0.7630360999691453 specificity=  0.9217922606924643 sensitivity=  0.7033560864618885 recall=  0.7033560864618885 MCC=  0.642200422886368 AUC=  0.9077927293095979 f1=  0.7319816486606482


(0.42307043597849653, 0.8642021595680864, 0.8125741735771764)

### esm2 1280 + MolFormer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_1280')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(2048, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_1280.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_1280.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_1280.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_1280.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_1280.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_1280.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_1280.pt')
    y_val = torch.load('ESP_val_df_label_esm2_1280.pt')
    y_test = torch.load('ESP_test_df_label_esm2_1280.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([50077, 1280]) torch.Size([5422, 1280]) torch.Size([13336, 1280])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3130/3130 [00:18<00:00, 168.70batch/s, train_loss=tensor(0.5457, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5702747650730223 ACC=  0.7366285503504242 bacc=  0.5113579278043663 precision=  0.5157894736842106 specificity=  0.9884740666499624 sensitivity=  0.03424178895877009 recall=  0.03424178895877009 MCC=  0.07631090941982721 AUC=  0.5974840140841001 f1=  0.06422018348623854
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5703); Accuracy (73.66)


Epoch 1: 100%|██████████| 3130/3130 [00:17<00:00, 174.18batch/s, train_loss=tensor(0.6738, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5422627621256144 ACC=  0.7401327923275544 bacc=  0.5088075528429533 precision=  0.84375 specificity=  0.9987471811576046 sensitivity=  0.018867924528301886 recall=  0.018867924528301886 MCC=  0.10136232466566568 AUC=  0.6545156371227296 f1=  0.03691045796308954
Epoch: 1 / 500, ############## the best accuracy in val  73.6629 at Epoch: 0  ##############
Performance in Val: Loss: (0.5423); Accuracy (74.01)


Epoch 2: 100%|██████████| 3130/3130 [00:18<00:00, 169.20batch/s, train_loss=tensor(0.4947, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.5114526716007413 ACC=  0.8959793434157137 bacc=  0.8502252009719282 precision=  0.8363072148952676 specificity=  0.9471310448509146 sensitivity=  0.753319357092942 recall=  0.753319357092942 MCC=  0.725235255856677 AUC=  0.9345380705469206 f1=  0.7926470588235294
Epoch: 497 / 500, ############## the best accuracy in val  89.8930 at Epoch: 485  ##############
Performance in Val: Loss: (0.5115); Accuracy (89.60)


Epoch 498: 100%|██████████| 3130/3130 [00:19<00:00, 163.31batch/s, train_loss=tensor(0.2611, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.4587721944860269 ACC=  0.8933972703799336 bacc=  0.8628151986273798 precision=  0.7980433263452131 specificity=  0.9275870709095465 sensitivity=  0.7980433263452131 recall=  0.7980433263452131 MCC=  0.7256303972547596 AUC=  0.9309250320558785 f1=  0.7980433263452131
Epoch: 498 / 500, ############## the best accuracy in val  89.8930 at Epoch: 485  ##############
Performance in Val: Loss: (0.4588); Accuracy (89.34)


Epoch 499: 100%|██████████| 3130/3130 [00:18<00:00, 171.66batch/s, train_loss=tensor(0.2083, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.4527898088426568 ACC=  0.8921062338620436 bacc=  0.8565592464246512 precision=  0.8043165467625899 specificity=  0.9318466549736908 sensitivity=  0.7812718378756115 recall=  0.7812718378756115 MCC=  0.7198710653709991 AUC=  0.9338001243538703 f1=  0.7926267281105991
Epoch: 499 / 500, ############## the best accuracy in val  89.8930 at Epoch: 485  ##############
Performance in Val: Loss: (0.4528); Accuracy (89.21)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_1280_simple_new.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.5352694592072978 ACC=  0.8758998200359928 bacc=  0.8321110477792529 precision=  0.7786762503743636 specificity=  0.924745417515275 sensitivity=  0.7394766780432309 recall=  0.7394766780432309 MCC=  0.675539020731937 AUC=  0.9187849869899372 f1=  0.7585703865791393


(0.5352694592072978, 0.8758998200359928, 0.8321110477792529)

In [ ]:
run_validation(model_test,val_loader,loss_fn, device) # performance evaluation

loss_sum=  0.46758457411281085 ACC=  0.898930284028034 bacc=  0.86388416564804 precision=  0.8206245461147422 specificity=  0.9381107491856677 sensitivity=  0.7896575821104123 recall=  0.7896575821104123 MCC=  0.7369323150478793 AUC=  0.9346475061550963 f1=  0.8048433048433048


(0.46758457411281085, 0.898930284028034, 0.86388416564804)

### esm2 2560 + MolFormer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(3328, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_simple_concatenate.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:17<00:00, 173.59batch/s, train_loss=tensor(0.5016, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.557613135618571 ACC=  0.7379144285978885 bacc=  0.5062094844559815 precision=  0.6470588235294118 specificity=  0.9969803724207348 sensitivity=  0.015438596491228071 recall=  0.015438596491228071 MCC=  0.06919625362322701 AUC=  0.5990096151299235 f1=  0.03015764222069911
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5576); Accuracy (73.79)


Epoch 1: 100%|██████████| 3118/3118 [00:17<00:00, 182.31batch/s, train_loss=tensor(0.5128, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5301314835276829 ACC=  0.7393961844786071 bacc=  0.5085663832454815 precision=  0.7368421052631579 specificity=  0.9974836436839456 sensitivity=  0.019649122807017545 recall=  0.019649122807017545 MCC=  0.09033044659526442 AUC=  0.6884780900414095 f1=  0.03827751196172249
Epoch: 1 / 500, ############## the best accuracy in val  73.7914 at Epoch: 0  ##############
Performance in Val: Loss: (0.5301); Accuracy (73.94)


Epoch 2: 100%|██████████| 3118/3118 [00:17<00:00, 179.00batch/s, train_loss=tensor(0.4983, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.5746079594913376 ACC=  0.9116503056121504 bacc=  0.8810193450410122 precision=  0.8439767779390421 specificity=  0.9458983392048314 sensitivity=  0.8161403508771929 recall=  0.8161403508771929 MCC=  0.7703830875724309 AUC=  0.9513262522183666 f1=  0.8298251872993222
Epoch: 497 / 500, ############## the best accuracy in val  91.7577 at Epoch: 495  ##############
Performance in Val: Loss: (0.5746); Accuracy (91.17)


Epoch 498: 100%|██████████| 3118/3118 [00:18<00:00, 171.87batch/s, train_loss=tensor(0.1815, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5816448388452626 ACC=  0.9099833302463419 bacc=  0.8789867471900688 precision=  0.8404641044234953 specificity=  0.9446401610468043 sensitivity=  0.8133333333333334 recall=  0.8133333333333334 MCC=  0.7660907577303916 AUC=  0.9495660389019857 f1=  0.8266761768901569
Epoch: 498 / 500, ############## the best accuracy in val  91.7577 at Epoch: 495  ##############
Performance in Val: Loss: (0.5816); Accuracy (91.00)


Epoch 499: 100%|██████████| 3118/3118 [00:17<00:00, 173.76batch/s, train_loss=tensor(0.0433, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5263775282202006 ACC=  0.9114650861270606 bacc=  0.8851696553916245 precision=  0.8341566690190544 specificity=  0.9408656265727227 sensitivity=  0.8294736842105264 recall=  0.8294736842105264 MCC=  0.7717344212309539 AUC=  0.9502166715227929 f1=  0.8318085855031669
Epoch: 499 / 500, ############## the best accuracy in val  91.7577 at Epoch: 495  ##############
Performance in Val: Loss: (0.5264); Accuracy (91.15)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_simple_concatenate.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_simple_new.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.7763116371656426 ACC=  0.8924278846153846 bacc=  0.8438345915799497 precision=  0.8325864276568502 specificity=  0.9466435421342583 sensitivity=  0.7410256410256411 recall=  0.7410256410256411 MCC=  0.7149743953105673 AUC=  0.9343656681495898 f1=  0.7841422972565572


(0.7763116371656426, 0.8924278846153846, 0.8438345915799497)